## Part 1: Assignment 2-A

## Data Preprocessing 

In [ ]:
import pandas as pd
import numpy as np

#### Question 1

In [ ]:
data="/Users/sofiagarfias/Documents/Machine Learning /ML_Assignment_ii_A_201920 (1)/kwb.csv"
df=pd.read_csv(data)

In [ ]:
#df.head()

In [ ]:
df = df[df['recs'] == 'Buurt']

#### Question 2

In [ ]:
df=df.drop(['gwb_code_10','gwb_code', 'gwb_code_8', 'regio', 'gm_naam', 'recs', 'ind_wbi'],axis=1)

#### Question 3

In [ ]:
df=df.replace('.', np.nan)

In [ ]:
df = df.dropna(subset=['g_ele','g_gas'])

In [ ]:
#df.head()

#### Question 4

In [ ]:
df_target=df[["g_ele","g_gas"]]

#### Question 5

In [ ]:
df_target.dtypes
df_target=df_target.astype("int64")
df_target["g_ele"]=pd.qcut(df_target["g_ele"],q=4,labels=False)
df_target["g_gas"]=pd.qcut(df_target["g_gas"],q=4,labels=False)
df=df.drop(['g_gas','g_ele'],axis=1)
df_target.isnull().sum()

* We still have missing values in our training dataset. However these will be imputed in the following question.


* The observations with missing values for the labels have been removed.

### Models

#### Question 1: Feature Imputations

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from scipy import interp
import matplotlib.pyplot as plt

In [ ]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    
    return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
# taken from scikit documentation
def binarize_getroc(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    n_classes = y_test.shape[1]
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    return fpr, tpr, roc_auc

gas_fpr = {}
gas_tpr = {}
gas_roc_auc = {}

ele_fpr = {}
ele_tpr = {}
ele_roc_auc = {}

### "g_gas" Models 

In [ ]:
X, y = df, df_target['g_gas']
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=42)

#### "g_gas" Unscaled Linear SVC model 

In [ ]:
linear_svc_non_scaled = make_pipeline(SimpleImputer(strategy='mean'), LinearSVC(C=1, random_state = 42))
linear_svc_non_scaled.fit(X_train, y_train)
linear_svc_non_scaled.score(X_test, y_test)
y_pred = linear_svc_non_scaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred))) 
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

#### "g_gas" Scaled Linear SVC model 

In [ ]:
linear_svc_scaled = make_pipeline(SimpleImputer(strategy='mean'), MinMaxScaler(), LinearSVC(C=1, random_state = 42))
linear_svc_scaled.fit(X_train, y_train)
linear_svc_scaled.score(X_test, y_test)
y_pred = linear_svc_scaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred))) 
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)


model_name = 'linear svc scaled'
gas_fpr[model_name], gas_tpr[model_name], gas_roc_auc[model_name] = binarize_getroc(y_test,y_pred)

* The confusion matrix is summarised using *precision* and *recall*.


* Our value for precision which measures how many of the samples predicted as positive are actually positive is:
    * **Unscaled SVC**: 0.72
    * **Scaled SVC**: 0.68
    * We can infer from this that even though the Unscaled SVC is higher they are quite similar and it is important to note that the Unscaled SVC did not converge. Thus, the valididty of this metric might be negatively affected and should not be compared to the Scaled score. 
     
     
* Our value for recall measures just how many of our samples were labeled correctly. A higher recall value shows that the model does not produce many false positives.
    * **Unscaled SVC**: 0.37
    * **Scaled SVC**: 0.69
    * From this score we can infer that scaling the model produces far better results. This is to be expected as the SVC is model is quite sensitive to scaling. 
        
        
* The f-score does a very good job of summarising both recall and precision. 
    * **Uscaled SVC**:0.32
    * **Scaled SVC**: 0.68
    * This clearly shows that in total the scaled model is superior.
        

#####  "g_gas"  Unscaled Logistic Regression Model

In [ ]:
log_reg_non_scaled = make_pipeline(SimpleImputer(strategy='mean'), LogisticRegression(solver="liblinear", multi_class = 'ovr', random_state = 42))
param_grid = {'logisticregression__C': [.001, 1, 100]}
grid = GridSearchCV(log_reg_non_scaled, param_grid, cv=10, n_jobs=-1)
grid.fit(X_train, y_train)
grid.score(X_test, y_test)
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))
print("Tuned Logistic Regression Parameters: {}".format(grid.best_params_))
print('Training accuracy:{}'.format(grid.score(X_train, y_train)))
print('Test accuracy:{}'.format(grid.score(X_test, y_test)))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

##### "g_gas" Scaled Logistic Regression Model

In [ ]:
log_reg_scaled = make_pipeline(SimpleImputer(strategy='mean'), MinMaxScaler(), LogisticRegression(solver="liblinear", multi_class = 'ovr', random_state = 42))
param_grid = {'logisticregression__C': [.001, 1, 100]}
grid = GridSearchCV(log_reg_scaled, param_grid, cv=10, n_jobs=-1)
grid.fit(X_train, y_train)
grid.score(X_test, y_test)
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))
print("Tuned Logistic Regression Parameters: {}".format(grid.best_params_))
print('Training accuracy:{}'.format(grid.score(X_train, y_train)))
print('Test accuracy:{}'.format(grid.score(X_test, y_test)))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

model_name = 'logistic regression scaled'
gas_fpr[model_name], gas_tpr[model_name], gas_roc_auc[model_name] = binarize_getroc(y_test,y_pred)

* The confusion matrix is summarised using *precision* and *recall*.

* Our value for precision which measures how many of the samples predicted as positive are actually positive is:
    * **Unscaled LR**: 0.71
    * **Scaled LR**: 0.70
    * We can infer from this Logistic Regression is not very sensitive to scaling which is why the precision metric has the same score. 
     
     
* Our value for recall measures just how many of our samples were labeled correctly. A higher recall value shows that the model does not produce many false positives.
    * **Unscaled LR**: 0.72
    * **Scaled LR**: 0.71
    * We can infer from this Logistic Regression is not very sensitive to scaling which is why the precision metric has the same score. 

        
* The f-score does a very good job of summarising both recall and precision. 
    * **Uscaled LR**:0.71
    * **Scaled LR**: 0.70
    * We can infer from this Logistic Regression is not very sensitive to scaling which is why the precision metric has the same score. 
        

### "g_ele" Models

In [ ]:
X, y = df, df_target['g_ele']
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=42)
## adding stratify=y increases the score of the non-scaled

#### "g_ele" Unscaled Linear SVC model 

In [ ]:
linear_svc_non_scaled = make_pipeline(SimpleImputer(strategy='mean'), LinearSVC(C=1, random_state = 42))
linear_svc_non_scaled.fit(X_train, y_train)
linear_svc_non_scaled.score(X_test, y_test)
y_pred = linear_svc_non_scaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred))) 
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

#### "g_ele" Scaled Linear SVC model 

In [ ]:
linear_svc_scaled = make_pipeline(SimpleImputer(strategy='mean'), MinMaxScaler(), LinearSVC(C=1, random_state = 42))
linear_svc_scaled.fit(X_train, y_train)
linear_svc_scaled.score(X_test, y_test)
y_pred = linear_svc_scaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred))) 
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

model_name = 'linear svc scaled'
ele_fpr[model_name], ele_tpr[model_name], ele_roc_auc[model_name] = binarize_getroc(y_test,y_pred)

* The confusion matrix is summarised using *precision* and *recall*.


* Our value for precision which measures how many of the samples predicted as positive are actually positive is:
    * **Unscaled SVC**: 0.48
    * **Scaled SVC**: 0.69
    * We can infer from this that even though the Unscaled SVC is higher they are quite similar and it is important to note that the Unscaled SVC did not converge. Thus, the valididty of this metric might be negatively affected and should not be compared to the Scaled score. 
     
     
* Our value for recall measures just how many of our samples were labeled correctly. A higher recall value shows that the model does not produce many false positives.
    * **Unscaled SVC**: 0.43
    * **Scaled SVC**: 0.69
    * From this score we can infer that scaling the model produces far better results. This is to be expected as the SVC is model is quite sensitive to scaling. 
      
      
* The f-score does a very good job of summarising both recall and precision. 
    * **Uscaled SVC**:0.37
    * **Scaled SVC**: 0.68
    * This clearly shows that in total the scaled model is superior.
        

#### "g_ele" Unscaled Logistic Regression Model

In [ ]:
log_reg_non_scaled = make_pipeline(SimpleImputer(strategy='mean'), LogisticRegression(solver="liblinear", multi_class = 'ovr', random_state = 42))
param_grid  = {'C': [.001, 1, 100]}
logreg_cv = GridSearchCV(log_reg_non_scaled, param_grid, cv=5,n_jobs=-1)
grid.fit(X_train, y_train)
grid.score(X_test, y_test)
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))
print("Tuned Logistic Regression Parameters: {}".format(grid.best_params_))
print('Training accuracy:{}'.format(grid.score(X_train, y_train)))
print('Test accuracy:{}'.format(grid.score(X_test, y_test)))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

#### "g_ele" Scaled Logistic Regression Model

In [ ]:
log_reg_scaled = make_pipeline(SimpleImputer(strategy='mean'), MinMaxScaler(), LogisticRegression(solver="liblinear", multi_class = 'ovr', random_state = 42))
param_grid = {'logisticregression__C': [.001, 1, 100]}
grid = GridSearchCV(log_reg_scaled, param_grid, cv=10, n_jobs=-1)
grid.fit(X_train, y_train)
grid.score(X_test, y_test)
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))
print("Tuned Logistic Regression Parameters: {}".format(grid.best_params_))
print("Best score is {}".format(grid.best_score_))
print('Training accuracy:{}'.format(grid.score(X_train, y_train)))
print('Test accuracy:{}'.format(grid.score(X_test, y_test)))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

model_name = 'logistic regression scaled'
ele_fpr[model_name], ele_tpr[model_name], ele_roc_auc[model_name] = binarize_getroc(y_test,y_pred)

* The confusion matrix is summarised using *precision* and *recall*.


* Our value for precision which measures how many of the samples predicted as positive are actually positive is:
    * **Unscaled LR**: 0.70
    * **Scaled LR**: 0.70
    * We can infer from this Logistic Regression is not very sensitive to scaling which is why the precision metric has the same score. 
    
    
* Our value for recall measures just how many of our samples were labeled correctly. A higher recall value shows that the model does not produce many false positives.
    * **Unscaled LR**: 0.70
    * **Scaled LR**: 0.70
    * We can infer from this Logistic Regression is not very sensitive to scaling which is why the recall metric has the same score. 

        
* The f-score does a very good job of summarising both recall and precision. 
    * **Uscaled LR**:0.70
    * **Scaled LR**: 0.70
    * We can infer from this Logistic Regression is not very sensitive to scaling which is why the f-score metric has the same score. 
        

##### Linear SVC Observations for "g_gas" and "g_ele"

As per the assignment we created a LinearSVC classifier. This classifier was applied to both target variables in an unscaled and scaled version. 

The scaling used was "MinMax" and it resulted in the following performance increases: 

* Our "g_gas" model had a performance increase of .33 after being scaled
* Our "g_ele" model had a performance increase of .27 after being scaled

We can conclude that preprocessing our data and making sure that it was scaled so that all features had values between 0 and 1 avoided certain features influencing the model more when compared to others. 

##### Logistic Regression Observations for "g_gas" and "g_ele"

From the best score results corresponding to "g_gas" and "g_ele" we can see how there is no effect of scaling on the Logistic Regression models. From this we can conclude that scaling our data had no impact on the outcome and scores of the Logistic Regression models. 

The "g_gas" model when regularized works best when C is set to 0.001. On the other hand the "g_ele" model works best when C is set to 100. 

The "g_ele" model corresponds to a model that stresses the importance of correctly classifying every point. Even though it results in a relatively more complex model it is able to extrapolate succesfully based on the very similar results both for the training and the testing. This possibly suggests undefitting. Even less regularization (higher values for C) could be explored.

From this we can conclude that regularization has improved generalization performance for both logistic regression models.

## Part 2: Assignment 2-B

### Decision Tree

#### "g_gas" Decision Tree Classifier

In [ ]:
X, y = df, df_target['g_gas']
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=42)

#### "g_gas" Unscaled Decision Tree Model

In [ ]:
decision_tree_unscaled = make_pipeline(SimpleImputer(strategy='mean'), DecisionTreeClassifier(random_state = 42))
decision_tree_unscaled.fit(X_train, y_train)
y_pred = decision_tree_unscaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
print("Accuracy on training set: {:.3f}".format(decision_tree_unscaled.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(decision_tree_unscaled.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

#### "g_gas" Scaled Decision Tree Model

In [ ]:
decision_tree_scaled = make_pipeline(SimpleImputer(strategy='mean'), MinMaxScaler(),DecisionTreeClassifier(random_state = 42))
decision_tree_scaled.fit(X_train, y_train)
y_pred = decision_tree_scaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
print("Accuracy on training set: {:.3f}".format(decision_tree_scaled.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(decision_tree_scaled.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

* Decision Trees are invariant to scaling of the data. 
* However, Decision Trees are can become very complex models and end up having pure leaves. As a result of this complexity they are very prone to overfitting and not being able to generalize on the testing set. However this can be prevented by using either pre-pruning or pruning strategies, another strategy is to make us of ensembles of Decision Trees.

* The confusion matrix is summarised using *precision* and *recall*.


* Our value for precision which measures how many of the samples predicted as positive are actually positive is:
    * **Unscaled DT**: 0.62
    * **Scaled DT**: 0.61
    * We can infer from this Decision Tree is not sensitive to scaling which is why the precision metric has aproximatley the same score. 
   
   
* Our value for recall measures just how many of our samples were labeled correctly. A higher recall value shows that the model does not produce many false positives.
    * **Unscaled DT**: 0.62
    * **Scaled DT**: 0.61
    * We can infer from this Decision Tree is not sensitive to scaling which is why the recall metric has aproximatley the same score. 


        
* The f-score does a very good job of summarising both recall and precision. 
    * **Uscaled DT**:0.62
    * **Scaled DT**: 0.61
    * We can infer from this Decision Tree is not sensitive to scaling which is why the F1 metrics have aproximatley the same score. 
        

#### Grid Search on "g_gas" model parameters

In [ ]:
tree = make_pipeline(SimpleImputer(strategy='mean'), MinMaxScaler(),DecisionTreeClassifier(random_state = 42))
param_grid = {'decisiontreeclassifier__max_depth': range(5,15,2), 
             'decisiontreeclassifier__max_features':range(40,59,5),
             'decisiontreeclassifier__min_samples_leaf':range(1,10,1),
             'decisiontreeclassifier__min_samples_split':[2,10,100]}
grid = GridSearchCV(tree, param_grid, cv=10, return_train_score = True, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)
print(grid.best_params_)
print(classification_report(y_test, y_pred))
print('Training accuracy:{}'.format(grid.score(X_train, y_train)))
print('Test accuracy:{}'.format(grid.score(X_test, y_test)))

model_name = 'decision tree'
gas_fpr[model_name], gas_tpr[model_name], gas_roc_auc[model_name] = binarize_getroc(y_test,y_pred)

#### "g_ele" Decision Tree Classifier

In [ ]:
X, y = df, df_target['g_ele']
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=42)

#### "g_ele" Unscaled Decision Tree Model

In [ ]:
decision_tree_unscaled = make_pipeline(SimpleImputer(strategy='mean'), DecisionTreeClassifier(random_state = 42))
decision_tree_unscaled.fit(X_train, y_train)
y_pred = decision_tree_unscaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
print("Accuracy on training set: {:.3f}".format(decision_tree_unscaled.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(decision_tree_unscaled.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

#### "g_ele" Scaled Decision Tree Model

In [ ]:
decision_tree_scaled = make_pipeline(SimpleImputer(strategy='mean'), MinMaxScaler(),DecisionTreeClassifier(random_state = 42))
decision_tree_scaled.fit(X_train, y_train)
y_pred = decision_tree_scaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
print("Accuracy on training set: {:.3f}".format(decision_tree_scaled.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(decision_tree_scaled.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

* The confusion matrix is summarised using *precision* and *recall*.


* Our value for precision which measures how many of the samples predicted as positive are actually positive is:
    * **Unscaled DT**: 0.61
    * **Scaled DT**: 0.61
    * We can infer from this Decision Tree is not sensitive to scaling which is why the precision metric has the same score. 
   
   
* Our value for recall measures just how many of our samples were labeled correctly. A higher recall value shows that the model does not produce many false positives.
    * **Unscaled DT**: 0.61
    * **Scaled DT**: 0.61
    * We can infer from this Decision Tree is not sensitive to scaling which is why the recall metric has the same score. 


        
* The f-score does a very good job of summarising both recall and precision. 
    * **Uscaled DT**:0.61
    * **Scaled DT**: 0.61
    * We can infer from this Decision Tree is not sensitive to scaling which is why the F1 metrics has the same score. 
        

#### Grid Search on "g_ele" model parameters

In [ ]:
tree = make_pipeline(SimpleImputer(strategy='mean'), MinMaxScaler(),DecisionTreeClassifier(random_state = 42))
param_grid = {'decisiontreeclassifier__max_depth': range(5,15,2), 
             'decisiontreeclassifier__max_features':range(40,59,5),
             'decisiontreeclassifier__min_samples_leaf':range(1,10,1),
             'decisiontreeclassifier__min_samples_split':[2,10,100]}
grid = GridSearchCV(tree, param_grid, cv=5, return_train_score = True, n_jobs=-1)
grid.fit(X_train, y_train)
y_pred = decision_tree_scaled.predict(X_test)

print(grid.best_params_)
print(classification_report(y_test, y_pred))
print("Best score is {}".format(grid.best_score_))
print('Training accuracy:{}'.format(grid.score(X_train, y_train)))
print('Test accuracy:{}'.format(grid.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

model_name = 'decision tree'
ele_fpr[model_name], ele_tpr[model_name], ele_roc_auc[model_name] = binarize_getroc(y_test,y_pred)

### Random Forest

#### "g_gas" Random Forest Classifier

In [ ]:
X, y = df, df_target['g_gas']
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=42)

#### "g_gas" Unscaled Random Forest

In [ ]:
random_forest_unscaled = make_pipeline(SimpleImputer(strategy='mean'), RandomForestClassifier(n_estimators = 100, random_state = 42))
random_forest_unscaled.fit(X_train, y_train)
random_forest_unscaled.score(X_test, y_test)
y_pred = random_forest_unscaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
print("Accuracy on training set: {:.3f}".format(random_forest_unscaled.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(random_forest_unscaled.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

#### "g_gas" Scaled Random Forest

In [ ]:
random_forest_scaled = make_pipeline(SimpleImputer(strategy='mean'),RandomForestClassifier(n_estimators = 100, random_state = 42))
random_forest_scaled.fit(X_train, y_train)
random_forest_scaled.score(X_test, y_test)
y_pred = random_forest_scaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
print("Accuracy on training set: {:.3f}".format(random_forest_scaled.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(random_forest_scaled.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

* The confusion matrix is summarised using *precision* and *recall*.

* Our value for precision which measures how many of the samples predicted as positive are actually positive is:
    * **Unscaled RF**: 0.71
    * **Scaled RF**: 0.71
    * We can infer from this Random Forest is not sensitive to scaling which is why the precision metric has the same score. 
     
     
* Our value for recall measures just how many of our samples were labeled correctly. A higher recall value shows that the model does not produce many false positives.
    * **Unscaled RF**: 0.72
    * **Scaled RF**: 0.72
    * We can infer from this Random Forest is not sensitive to scaling which is why the recall metric has the same score. 

        
* The f-score does a very good job of summarising both recall and precision. 
    * **Uscaled RF**:0.71
    * **Scaled RF**: 0.71
    * We can infer from this Random Forest is not sensitive to scaling which is why the F1 metric has the same score. 
        

#### Grid Search on "g_gas" model parameters

In [ ]:
rfc = make_pipeline(SimpleImputer(strategy='mean'), MinMaxScaler(),RandomForestClassifier(random_state = 42))
param_grid = {'randomforestclassifier__n_estimators': [10,100,200],
              'randomforestclassifier__max_features': ['auto', 'sqrt', 'log2'],
              'randomforestclassifier__max_depth' : [25,50,75],
              'randomforestclassifier__min_samples_leaf' : [1,3,5],
              'randomforestclassifier__min_samples_split' : [2,5,10]}
grid = GridSearchCV(rfc, param_grid, cv=5, return_train_score = True, n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_params_

y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy on training set: {:.3f}".format(grid.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(grid.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

model_name = 'random forest'
gas_fpr[model_name], gas_tpr[model_name], gas_roc_auc[model_name] = binarize_getroc(y_test,y_pred)

#### "g_ele" Random Forest Classifier

In [ ]:
X, y = df, df_target['g_ele']
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=42)

#### "g_ele" Unscaled Random Forest

In [ ]:
random_forest_unscaled = make_pipeline(SimpleImputer(strategy='mean'), RandomForestClassifier(n_estimators = 100, random_state = 42))
random_forest_unscaled.fit(X_train, y_train)
random_forest_unscaled.score(X_test, y_test)
y_pred = random_forest_unscaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
print("Accuracy on training set: {:.3f}".format(random_forest_unscaled.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(random_forest_unscaled.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

#### "g_ele" Scaled Random Forest

In [ ]:
random_forest_scaled = make_pipeline(SimpleImputer(strategy='mean'),MinMaxScaler(),RandomForestClassifier(n_estimators = 100, random_state = 42))
random_forest_scaled.fit(X_train, y_train)
random_forest_scaled.score(X_test, y_test)
y_pred = random_forest_unscaled.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, y_pred)))
print("Accuracy on training set: {:.3f}".format(random_forest_scaled.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(random_forest_scaled.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

* The confusion matrix is summarised using *precision* and *recall*.

* Our value for precision which measures how many of the samples predicted as positive are actually positive is:
    * **Unscaled RF**: 0.73
    * **Scaled RF**: 0.73
    * We can infer from this Random Forest is not sensitive to scaling which is why the precision metric has the same score. 
     
     
* Our value for recall measures just how many of our samples were labeled correctly. A higher recall value shows that the model does not produce many false positives.
    * **Unscaled RF**: 0.73
    * **Scaled RF**: 0.73
    * We can infer from this Random Forest is not sensitive to scaling which is why the recall metric has the same score. 

        
* The f-score does a very good job of summarising both recall and precision. 
    * **Uscaled RF**:0.73
    * **Scaled RF**: 0.73
    * We can infer from this Random Forest is not sensitive to scaling which is why the F1 metric has the same score. 
        

* Random Forests are also completely invariant to scaling of the data.

#### How are decision tree classifiers different from random forests on a structural level? 
##### Discuss their advantages/disadvantages over each other and where would you choose one over the other?

* Random Forests are different from Decision Trees on a strcutural level because Random Forests are an ensemble of unique and random Decision Trees. This allows us to reduce the level of overfitting by creating an average of the aggregated individual Decision Trees that make up the forest. Random Forests are also randomly created by bootstrapping the data which creates a significant structural difference when compared to Decision Trees. 


* However, given that Random Forests are created using multiple Decision Trees this produces models that are very complex and not suited for compact visual representations; this simpler visualization can be achieved with a Decision Tree model on the other hand. Both models work without scaling the data and do not require heavy tuning of parameters.


* For simple to understand models Decision Trees are more suitable, however if this is not an issue we would use a Random Forest. 



#### Grid Search on "g_ele" model parameters

In [ ]:
rfc = make_pipeline(SimpleImputer(strategy='mean'),MinMaxScaler(),RandomForestClassifier(random_state = 42))
param_grid = {'randomforestclassifier__n_estimators': [10,100,200],
              'randomforestclassifier__max_features': ['auto', 'sqrt', 'log2'],
              'randomforestclassifier__max_depth' : [25,50,75],
              'randomforestclassifier__min_samples_leaf' : [1,3,5],
              'randomforestclassifier__min_samples_split' : [2,5,10]}
grid = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_params_

y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))
print("Accuracy on training set: {:.3f}".format(grid.score(X_train, y_train))) 
print("Accuracy on test set: {:.3f}".format(grid.score(X_test, y_test)))
auc_score = multiclass_roc_auc_score(y_test, y_pred)
print("The AUC score for this model is:", auc_score)

model_name = 'random forest'
ele_fpr[model_name], ele_tpr[model_name], ele_roc_auc[model_name] = binarize_getroc(y_test,y_pred)

### ROC Curves
#### ROC Curves for "g_gas"

In [ ]:
# Plot all ROC curves for gas
gas = plt.figure()
gaseax = gas.add_subplot(1, 1, 1)

for model_name in gas_fpr:
    gaseax.plot(gas_fpr[model_name]["marco"], gas_tpr[model_name]["macro"], label='macro-average ROC curve for {:s} (area = {:0.2f})'.format(model_name, gas_roc_auc[model_name]["macro"]))

gaseax.plot([0, 1], [0, 1], 'k--', lw=2)
gaseax.set_xlim([0.0, 1.0])
gaseax.set_ylim([0.0, 1.05])
gaseax.set_xlabel('False Positive Rate')
gaseax.set_ylabel('True Positive Rate')
gaseax.set_title('Receiver operating characteristic - gas')
gas.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

#### ROC Curves for "g_ele"

In [ ]:
# Plot all ROC curves for electricity
ele = plt.figure()
eleeax = ele.add_subplot(1, 1, 1)

for model_name in ele_fpr:
    eleeax.plot(ele_fpr[model_name]["macro"], ele_tpr[model_name]["macro"],label='macro-average ROC curve for {:s} (area = {:0.2f})'.format(model_name, ele_roc_auc[model_name]["macro"]))

eleeax.plot([0, 1], [0, 1], 'k--', lw=2)
eleeax.set_xlim([0.0, 1.0])
eleeax.set_ylim([0.0, 1.05])
eleeax.set_xlabel('False Positive Rate')
eleeax.set_ylabel('True Positive Rate')
eleeax.set_title('Receiver operating characteristic - electricity')
ele.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

#### Is accuracy well suited to these classifications task?

* Accuracy is not well suited because the samples are not balanced, therefore ROC-AUC is preferable. 

#### Can we find the optimal model?

* Yes, we can select the optimal model. 
    * The optimal model for this classification task is the Random Forest. This can be seen from both ROC graphs and their respective AUC scores, that were printed for each model. Logisitc Regression and Linear SVC can also be considered a good alternative to the Random Forest model, and require significantly less computing power. 